# COSI140b Lab 3: A Brief Tutorial on Machine Learning with NLTK/Python
Keigh Rim, 4/12/2016


## Intro

This is a tutorial on machine learning techniques using NLTK in Python. It is based on Python 2.7 and NLTK3. 

The tutorial is about 
* transforming documents into feature representation
* training existing machine learning algorithms in NLTK
* testing the trained algorithms 

It is not about 
* engineering features
* theoretical aspects of machine learning

Now, let's start with importing nltk.

In [5]:
import nltk

## Supervised Classification

Here we will write Python code to train and test out a classifier using NLTK packages.

Our example task would be training a Naive Bayes classifier that predicts gender of the author given a blog post. 
As our dataset, we will use [`blog-gender-dataset`](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html). For details, refer to the paper;
* Mukherjee, A., & Liu, B. (2010, October). Improving gender classification of blog authors. In Proceedings of the 2010 conference on Empirical Methods in natural Language Processing (pp. 207-217). Association for Computational Linguistics.

In the dataset, we have about 3k blog posts, their raw unicode texts and their labels ('F', 'M') indicating the authors' gender. Let's take a peek at it. (The file is in CSV format.)


In [6]:
import csv

with open('blog-gender-dataset.csv') as dataset: 
    for i, row in enumerate(csv.reader(dataset)):
        print(row)
        if i > 0: break

[" Long time no see. Like always I was rewriting it from scratch a couple of times. But nevertheless it's still java and now it uses metropolis sampling to help that poor path tracing converge.\n\nBtw. I did MLT on yesterday evening after 2 beers (it had to be Ballmer peak).\n\nAltough the implementation is still very fresh it easily outperforms standard path tracing, what is to be seen especially when difficult caustics are involved.\n\nI've implemented spectral rendering too, it was very easy actually, cause all computations on wavelengths are linear just like rgb. But then I realised that even if it does feel more physically correct to do so, whats the point? 3d applications are operating in rgb color space, and because I cant represent a rgb color as spectrum interchangeably I have to approximate it, so as long as I'm not running a physical simulation or something I don't see the benefits (please correct me if I'm wrong), thus I abandoned that.", 'M']
[' Guest Demo: Eric Iverson\xe

These are our *raw* data.

### Feature Representation

So, as saw above, our *raw* dataset is simply a set of <**unicode string**, **label**> pairs.

Now we are turning this into a <**features**, **label**> set. That is, we will write this piece of code. 

In [7]:
def normalize_label(label):
    return label.strip().upper()
    
def extract_features(text):
    raise NotImplementedError
    
with open('blog-gender-dataset.csv') as dataset: 
    raw_data = {raw_text : normalize_label(label) 
                for raw_text, label in csv.reader(dataset)}
    feature_representation = {extract_features(text) : label 
                              for text, label in raw_data.iteritems() 
                              if label} # some items are missing label

NotImplementedError: 

#### Defining features
Then, what is a feature? A feature is simply a fragmentary/single-faceted description of an instance/item in the data. It can be anything, for example: 
* *Does the document have a proper name in it? - **Yes***  (Binary feature)
* *How many characters are appearing in the document? - **3***  (Discrete numerical feature)
* *What is the most frequent non-stopword in the document? - **'time'*** (Nominal feature)

A set of features is exactly the way we want to describe an item. And the set of feature values for an item is the representation of that item in the model we design. Thus, after all, we are transforming a document into a set of name-value pairs.

However, to statistically model the data, each description (feature value) **needs to be a number**! Binary and numerical features are inherently convertible to numbers. Nominal or ordinal features can be mapped to numbers using random variables. Designing such random variables is beyond the scope of this tutorial and, for the sake of simplicity, we will be using only binary features in the rest of this section. 



#### Getting feature values
Next question: how can we get values of the features? For instance, say we have a binary feature that represents whether a document contains the word 'time'. How do we get the value for it? We write a feature function to get the answer. 

In [9]:
def has_time(document):
    # returns a feature (yes | no) value directly
    return 'time' in document.split()  

Besides of the `has_time()` function, suppose we also have `has_flies()`, `has_like()`, `has_an()`, and `has_arrow()` functions. 

In [10]:
def has_flies(document):
    return 'flies' in document.split()
def has_like(document):
    return 'like' in document.split()
def has_an(document):
    return 'an' in document.split()
def has_arrow(document):
    return 'arrow' in document.split()

Then these two short documents are represented as follows in the feature space. 

In [11]:
document1 = 'time flies like an arrow'
document2 = 'time flies when you are having fun'

doc1_features = {'has_time' : has_time(document1) ,
                'has_flies' : has_flies(document1) ,
                'has_like' : has_like(document1) ,
                'has_an' : has_an(document1) ,
                'has_arrow' : has_arrow(document1)}
doc2_features = {'has_time' : has_time(document2) ,
                'has_flies' : has_flies(document2) ,
                'has_like' : has_like(document2) ,
                'has_an' : has_an(document2) ,
                'has_arrow' : has_arrow(document2)}
print 'document1: ', doc1_features
print 'document2: ', doc2_features

document1:  {'has_an': True, 'has_like': True, 'has_arrow': True, 'has_time': True, 'has_flies': True}
document2:  {'has_an': False, 'has_like': False, 'has_arrow': False, 'has_time': True, 'has_flies': True}


Then, our `extract_features()` functions can be written like this.

In [12]:
feature_functions = [has_time, 
                     has_flies, 
                     has_like, 
                     has_an, 
                     has_arrow ] 

def extract_features(document):
    return {feature_function.__name__: feature_function(document) 
            for feature_function in feature_functions}

print 'document1: ', extract_features(document1)

document1:  {'has_an': True, 'has_like': True, 'has_arrow': True, 'has_time': True, 'has_flies': True}


Okay, that was a silly example, now let's consider a more realistic example using `blog-gender-dataset`. 

Let's say we want to use all the unigrams as our features, using simple whitespace tokenization. Then we are going to have ...

In [13]:
dataset_file = open('blog-gender-dataset.csv')
raw_data = {raw_text: normalize_label(label) 
            for raw_text, label in csv.reader(dataset_file) if label}
dataset_file.close()

vocabulary = set()
for document in raw_data.keys():
    vocabulary.update(document.split())
print(len(vocabulary))

128077


Boom! We are going to have 128k features. So, do we need to write .1 million feature functions? Of course not. We can re-write the `extract_features()` function, to avoid writing 128k other functions, like such: 

In [14]:
def extract_features(document):
    # now each feature_function can handle multiple features
    # and should return the values wrapped in a dict
    features = {}
    for feature_function in feature_functions:
        features.update(feature_function(document))  
    return features

def unigram_bow(document):
    return {token: (token in document.split()) for token in vocabulary}
    
feature_functions = [unigram_bow]

Now, how can you write another feature function `bigram_bow()` and incorporate it into `extract_features()`? And then how many features are we going to have? If we iterate through all three thousands of documents in the dataset and compute all the features, how long would it take? Can you write something in a more efficient way? For example, are all the word types including stopwords in the corpus going to be helpful features? How about stemming and lemmaization?

#### Feature representation for NLTK classifiers
Naive Bayes and MaxEnt in NLTK takes a list of tuples (feature_representation, label) as a training set. And they are smart enough to automatically take non-specified features as having *null*  values (`False`, `None`, `0`, etc) by default. As a result, for example, to get `unigram_bow()` features, we don't need to iterate over the whole vocabulary over and over. Rather, we re-write the function:

In [15]:
def unigram_bow_rewrite(document):
    return {token: True for token in document.split()}

dataset_file = open('blog-gender-dataset.csv')
raw_data = [(raw_text, normalize_label(label))  
            for raw_text, label in csv.reader(dataset_file) if label] 
# lazy evaluation might be a better idea in real implementation
            
dataset_file.close()

feature_functions = [unigram_bow_rewrite]
print(extract_features(raw_data[0][0]))

{'wavelengths': True, 'all': True, '(it': True, 'help': True, 'just': True, 'represent': True, 'color': True, 'scratch': True, 'too,': True, 'sampling': True, 'actually,': True, 'rendering': True, 'But': True, 'times.': True, 'see.': True, 'whats': True, 'Like': True, 'still': True, 'physical': True, 'feel': True, 'from': True, "I've": True, 'implementation': True, "it's": True, "don't": True, 'had': True, 'approximate': True, 'long': True, 'to': True, 'rgb': True, '2': True, 'easy': True, 'especially': True, 'was': True, 'more': True, 'then': True, 'it,': True, 'evening': True, 'that': True, 'very': True, 'Btw.': True, 'couple': True, 'Altough': True, 'space,': True, 'so,': True, 'cant': True, 'it': True, 'beers': True, 'difficult': True, 'not': True, "I'm": True, 'now': True, 'abandoned': True, 'me': True, 'easily': True, 'like': True, 'peak).': True, 'did': True, 'always': True, 'as': True, 'tracing,': True, 'wrong),': True, 'computations': True, 'of': True, '3d': True, 'even': True

Putting all together, we finally translate the entire dataset into `feature_representation`. 

In [16]:
feature_representation = [(extract_features(raw_text), label) 
                          for raw_text, label in raw_data]

Lastly, let's split the dataset into the training set and the test set for testing the algorithm.

In [38]:
testset_percentage = 10

def split_dataset(dataset, testset_percentage):
    cutoff = testset_percentage * len(dataset) / 100
    return dataset[cutoff:], dataset[:cutoff]

trainset, testset = split_dataset(feature_representation, 
                                  testset_percentage)

### Training classifiers

Once we have the feature representation, training a classifier is straightforward. We only need to call `train()` method on our data. 


In [18]:
nb_classifier = nltk.classify.NaiveBayesClassifier.train(trainset)

Other than Naive Bayes, NLTK comes with MaxEnt, decision tree, and many other classifier implementations. For details, refer to [the official documentation](http://www.nltk.org/api/nltk.classify.html). 

### Testing a classifier

Now let's see how good our naive bayes classifier is. After training, the classifier can perform classification upon `classify()` calls. 

In [37]:
predictions = [nb_classifier.classify(test_document) 
               for test_document, _ in testset]

ValueError: too many values to unpack

We can measure its performance on many different aspect. First, let's see its accuracy.

In [39]:
def accuracy(predictions, golds):
    return (len([p for p, g in zip(predictions, golds) if p == g]) 
            / float(len(golds)))

golds = [label for _ , label in testset]
print(accuracy(predictions, golds))

0.711180124224


To compute precision and recall, NLTK provides a convenient method to draw a confusion matrix as well as computation of precision and recall. However, we need a different data structure; sets of document id's of different labels. 

In [21]:
c_matrix = nltk.ConfusionMatrix(golds, predictions)
print c_matrix

gold_sets = {"M": set(), "F": set()}
pred_sets = {"M": set(), "F": set()}
for doc_id, label in enumerate(golds): 
    gold_sets[label].add(doc_id)
for doc_id, label in enumerate(predictions): 
    pred_sets[label].add(doc_id)

from nltk.metrics import scores
for label in ['M', 'F']:
    r = scores.recall(gold_sets[label], pred_sets[label])
    p = scores.precision(gold_sets[label], pred_sets[label])
    f = scores.f_measure(gold_sets[label], pred_sets[label])
    print('<{}> P: {:.2}, R: {:.2}, F: {:.2}'.format(label, p, r, f))

  |   F   M |
--+---------+
F | <92> 36 |
M |  57<137>|
--+---------+
(row = reference; col = test)

<M> P: 0.79, R: 0.71, F: 0.75
<F> P: 0.62, R: 0.72, F: 0.66


Lastly, but definately not least, NLTK provides a method to rank the helpfulness of features.

In [22]:
nb_classifier.show_most_informative_features(10)

Most Informative Features
                 devices = True                M : F      =     13.1 : 1.0
                   users = True                M : F      =     12.8 : 1.0
                  notion = True                M : F      =     12.4 : 1.0
                    sun. = True                F : M      =     12.2 : 1.0
                      ME = True                F : M      =     11.5 : 1.0
                fabulous = True                F : M      =     10.2 : 1.0
                  topped = True                F : M      =     10.1 : 1.0
                   Bruce = True                M : F      =      9.9 : 1.0
            technologies = True                M : F      =      9.9 : 1.0
                    too! = True                F : M      =      9.8 : 1.0


So far, we have seen how to write Python code to transform documents into feature sets and to train/test a classifier for a simple classification problem. Now let's take a look at different types of problems.

## Clustering

### k_means clustering

NLTK clustering implementations requires a **complete vector representation** (using `numpy.ndarray`) of the corpus, not a `dict` based featuresets.

Let's do this with 128k unigram features, which will result in a very sparse feature vector. 

In [ ]:
import numpy 

feature_indices = list(vocabulary) # we need an indexed feature names

feature_vectors = []   # this will be a #documents X #features matrix
                       # a huge and sparse one
    
def generate_document_vector(document):
    document_vector = numpy.zeros(len(feature_indices))
    for word_type in set(document.split()):
        document_vector[feature_indices.index(word_type)] = 1
    return document_vector

feature_vectors = [generate_document_vector(document) 
                   for document, _ in raw_data[:10]]

By the way, [`numpy.ndarray`](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.ndarray.html) is a matrix-like python object that is highly optimized for numerical computation. Linear algebraic operations, such as arithmetic on matrices or matrix manipulations, are super fast with ndarrays, outperforming list comprehensions in Python with a huge gap (not to mention looping), so most scientific packages in Python including all widely-used machine learning packages (scikit-learn, theano, tensorflow, you name it) are heavily relying on `numpy` data structures and functions. 

Next, we will build 2 clusters using k-means algorithm based on cosine similarity between these vectors. Note that the k-means algorithm starts with random seeds and does not guarantee the global convergence. Thus, one might want to repeat the algorithm then take the most common result as a 'good enough' clustering (by using `repeats` parameter).

In [ ]:
dist = nltk.cluster.cosine_distance
kmc = nltk.cluster.kmeans.KMeansClusterer(2, dist, repeats=10)

clustered = kmc.cluster(feature_vectors, True)
print(clustered)

gold_labels = [int(label=='M') for _, label in raw_data[:10]]
print(gold_labels)

After trained, the clustering can be used a sort-of classifier, like such:

In [ ]:
kmc.classify(numpy.array(generate_document_vector(raw_data[111][0])))

## Supervised Sequencial Tagging

### HMM tagger

Here, we are going to train a HMM sequence tagger for Chinese word segmentation, using **BIO tagging**. Included in the directory is word segmentation annotation of around 20k Chinese sentences from news articles, a small part of [Chinese Treebank corpus](http://www.cs.brandeis.edu/~clp/ctb/). 
Let's take a look at the annotation.

In [44]:
with open('tagged.seg') as seg_annotation:
    for i, line in enumerate(seg_annotation):
        print line
        if i > 1: break

最_B 近_O ，_B 我_B 们_O 通_B 过_O 知_B 情_O 人_B 士_O 从_B 衡_B 阳_I 市_O 殡_B 葬_O 管_B 理_I 处_O 财_B 务_O 部_B 门_O 复_B 印_O 出_B 部_B 分_O 原_B 始_O 发_B 票_O 凭_B 证_O 1_B 1_O 份_B （_B 共_B 计_O 有_B 3_B 0_I 余_O 份_B ）_B 。_B

在_B 这_B 些_O 原_B 始_O 凭_B 证_O 上_B ，_B 我_B 们_O 可_B 以_O 清_B 清_I 楚_I 楚_O 地_B 看_B 到_O 如_B 此_O 的_B 批_B 示_O ：_B “_B 招_B 待_O 省_B 高_B 院_O 民_B 三_I 庭_O 法_B 官_O 来_B 衡_B 阳_O 调_B 查_O 取_B 证_O 餐_B 费_O ”_B 、_B “_B 到_B 省_B 高_B 院_O 协_B 调_O 与_B 明_B 田_O 公_B 司_O 诉_B 讼_O 问_B 题_O ，_B 招_B 待_O 上_B 级_O 领_B 导_O ”_B 、_B “_B 局_B 领_B 导_O 去_B 高_B 院_O ，_B 招_B 待_O 费_B 用_O ”_B 、_B “_B 招_B 待_O 省_B 高_B 院_O 领_B 导_O ”_B 、_B “_B 省_B 高_B 院_O 到_B 衡_B 阳_O 协_B 调_O 我_B 单_B 位_O 与_B 明_B 田_O 公_B 司_O 合_B 同_O 纠_B 纷_O 一_B 案_B ”_B 等_B 等_O （_B 证_B 据_O 复_B 印_I 件_O 附_B 后_B ）_B 。_B

这_B 些_O 原_B 始_O 发_B 票_O 凭_B 证_O 足_B 以_O 证_B 明_O ，_B 这_B 是_B 一_B 起_B 典_B 型_O 的_B 司_B 法_O 不_B 公_O 、_B 司_B 法_O 腐_B 败_O 案_B ，_B 铁_B 证_I 如_I 山_O ！_B



**Disclaimer**: For details for CTB, see the paper: 
* Xue, N., Xia, F., Chiou, F. D., & Palmer, M. (2005). The Penn Chinese TreeBank: Phrase structure annotation of a large corpus. Natural language engineering, 11(02), 207-238.

The original word segmentation is done with simple whitespace, and I did BIO tagging based on the annotation last night. It was a complete hack job, and all errors in BIO tagging are my fault. 

To feed the data into supervised training process implemented in NLTK, we need to prepare a sequence of <**observated, state**> tuples.

In [24]:
tagged_segmentation = []
with open('tagged.seg') as annotation:
    for line in annotation:
        tagged_segmentation.append(
            [tuple(token.split("_")) for token in line.split()])
    
print(len(tagged_segmentation))
print(tagged_segmentation[234])



19926
[('\xe5\x8d\x81', 'B'), ('\xe4\xb8\x89', 'I'), ('\xe4\xba\xbf', 'O'), ('\xe6\xb0\x91', 'B'), ('\xe8\x8b\xa6', 'B'), ('\xe7\x85\x8e', 'B'), ('\xe7\x86\xac', 'O')]


Okay, now we have about 20k sequences of word segmentation tagging. As always, training starts with split the data into the train set and the test set.

In [41]:
trainset, testset = split_dataset(tagged_segmentation, 
                                  testset_percentage)  # = 10

Again, training is fairly straightforward. We need to create an trainer object and then call `train()` method.

In [26]:
from nltk.tag.hmm import HiddenMarkovModelTrainer
hmm_tagger = HiddenMarkovModelTrainer().train(trainset)

After all, we now have a tagger that can perform word segmentation on an arbitrary Chinese sentence. Let's try.

In [54]:
def glue_chars(chars_seq):
    return "".join(chars_seq)

def bio_to_whtspc(tagged_seq):
    return glue_chars([" " + char if tag == "B" else char 
                       for char, tag in tagged]).strip()

print "ori:", glue_chars([char for char, tag in testset[66]])
print "gld:", bio_to_whtspc(testset[66])
print "tag:", bio_to_whtspc(tagged_segmentation[66])

ori: 他们想着陶家会‘积极’赔偿履行诺言，陪同着孩子一直治疗直到好为止。
gld: 他们 想 着 陶家 会 ‘ 积极 ’ 赔偿 履 行诺 言 ， 陪同 着 孩子 一直 治疗 直到 好 为止 。
tag: 他们 想 着 陶家 会 ‘ 积极 ’ 赔偿 履 行诺 言 ， 陪同 着 孩子 一直 治疗 直到 好 为止 。


But, how good does it perform in general? We can measure the tagger's accuracy with a simple computation, as we did in the above. 

In [ ]:
predictions= []
gold = []

for sent in testset:
    predictions.extend([predicted for _, predicted 
                        in hmm_tagger.tag([char for char, tag in sent])])
    gold.extend([gold_tag for _, gold_tag in sent])

print "accuracy:", accuracy(predictions, gold)

As well as confusion matrix, and P/R/F measures.

In [32]:
print(nltk.ConfusionMatrix(gold, predictions))    

  |     B     I     O |
--+-------------------+
B |<38122>  553  5850 |
I |  1448  <527>  794 |
O |  5253   199<14093>|
--+-------------------+
(row = reference; col = test)

